In [1]:
import numpy as np
import pandas as pd
# import sklearn

input_file = "kc_house_data.csv"
df = pd.read_csv(input_file)

In [2]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21613 entries, 735 to 15795
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long

In [4]:
corr_matrix=df.corr()

In [5]:
corr_matrix["price"].sort_values(ascending=False)

price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
floors           0.256794
yr_renovated     0.126434
sqft_lot         0.089661
sqft_lot15       0.082447
yr_built         0.054012
condition        0.036362
long             0.021626
id              -0.016762
zipcode         -0.053203
Name: price, dtype: float64

In [6]:
df['total_rooms'] = df['sqft_living']+ df['bedrooms'] + df['bathrooms']
df['sqft_total_size'] = df['sqft_living'] + df['sqft_above'] + df['sqft_basement']
df['yard'] = df['sqft_lot']-df['sqft_living']
#df['b'] = df["sqft_living"] / df['sqft_living15']
df['d'] = df['grade'] * df['condition']
df['total_condition']=df['condition']/df['yr_built'] 
df["outsideview"] = df["waterfront"]+df["lat"]
df["location"]=df['long']*df['lat']
df["outsideview2"] = df["waterfront"]*df["view"]+df['lat']
df["grade^2"]=df["grade"]**2
df["long*view"]=df["long"]*df["view"]


In [7]:
housing_prepared = df.drop((["price","date","yr_renovated","bathrooms"]),axis=1)
housing_labels = df["price"].copy()


In [8]:
corr_matrix=df.corr()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21613 entries, 735 to 15795
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               21613 non-null  int64  
 1   date             21613 non-null  object 
 2   price            21613 non-null  float64
 3   bedrooms         21613 non-null  int64  
 4   bathrooms        21613 non-null  float64
 5   sqft_living      21613 non-null  int64  
 6   sqft_lot         21613 non-null  int64  
 7   floors           21613 non-null  float64
 8   waterfront       21613 non-null  int64  
 9   view             21613 non-null  int64  
 10  condition        21613 non-null  int64  
 11  grade            21613 non-null  int64  
 12  sqft_above       21613 non-null  int64  
 13  sqft_basement    21613 non-null  int64  
 14  yr_built         21613 non-null  int64  
 15  yr_renovated     21613 non-null  int64  
 16  zipcode          21613 non-null  int64  
 17  lat       

In [10]:
corr_matrix["price"].sort_values(ascending=False)

price              1.000000
sqft_total_size    0.702035
sqft_living        0.702035
total_rooms        0.701933
grade^2            0.692402
grade              0.667434
sqft_above         0.605567
sqft_living15      0.585379
bathrooms          0.525138
d                  0.490904
outsideview        0.404086
view               0.397293
outsideview2       0.366842
sqft_basement      0.323816
bedrooms           0.308350
lat                0.307003
waterfront         0.266369
floors             0.256794
yr_renovated       0.126434
sqft_lot           0.089661
sqft_lot15         0.082447
yard               0.074361
yr_built           0.054012
condition          0.036362
total_condition    0.032805
long               0.021626
id                -0.016762
zipcode           -0.053203
location          -0.265504
long*view         -0.397268
Name: price, dtype: float64

In [11]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'total_rooms',
       'sqft_total_size', 'yard', 'd', 'total_condition', 'outsideview',
       'location', 'outsideview2', 'grade^2', 'long*view'],
      dtype='object')

In [12]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [32]:
from sklearn.ensemble import ExtraTreesRegressor

rnd_reg = ExtraTreesRegressor(n_estimators=1000, random_state=42,max_depth=30)
rnd_reg.fit(housing_prepared, housing_labels)

ExtraTreesRegressor(max_depth=30, n_estimators=1000, random_state=42)

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesRegressor

rnd_reg = ExtraTreesRegressor(n_estimators=1000, random_state=42,max_depth=30)
rnd_reg.fit(housing_prepared, housing_labels)
temp = []
model = [rnd_reg]
def cv10(model, housing_prepared, housing_labels):
    model.fit(housing_prepared, housing_labels)
    model_scores = cross_val_score(model, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
    model_rmse_scores = np.sqrt(-model_scores)
    temp.append(model_rmse_scores)


In [34]:
for i in model:
    scores = cv10(i, housing_prepared,housing_labels)


In [35]:
pd.Series(*temp).mean()

nan

# Stacking

In [17]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

estimators = [
     ('ex', ExtraTreesRegressor(n_estimators=1000, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           SVR(kernel="rbf",gamma="scale",degree=18,coef0=100,C=50)))
 ]
reg = StackingRegressor(
     estimators=estimators, final_estimator= ExtraTreesRegressor(n_estimators=600,max_depth=30, random_state=42,n_jobs=-1)
)
reg.fit(housing_prepared, housing_labels)


StackingRegressor(estimators=[('ex',
                               ExtraTreesRegressor(n_estimators=400,
                                                   random_state=42)),
                              ('svr',
                               Pipeline(steps=[('standardscaler',
                                                StandardScaler()),
                                               ('svr',
                                                SVR(C=50, coef0=100,
                                                    degree=18))]))],
                  final_estimator=ExtraTreesRegressor(max_depth=30,
                                                      n_estimators=600,
                                                      n_jobs=-1,
                                                      random_state=42))

In [20]:
reg_scores = cross_val_score(reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10,n_jobs=-1)
score = np.sqrt(-reg_scores)
display_scores(score)

Scores: [150820.14695008 162984.4673912  141549.5635203  161638.18756682
 146489.43164253 125713.75796903 129585.45054999 127120.25001896
 134118.75947174 125959.11289971]
Mean: 140597.9127980352
Standard deviation: 13655.381237315203


# voting

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor

ex1_reg = ExtraTreesRegressor(n_estimators=1000, random_state=42,max_depth=30)
ex1_reg.fit(housing_prepared, housing_labels)
ex2_reg = ExtraTreesRegressor(n_estimators=800, random_state=42,max_depth=60)
ex2_reg.fit(housing_prepared, housing_labels)
ex3_reg = ExtraTreesRegressor(n_estimators=500, random_state=42,max_depth=30)
ex3_reg.fit(housing_prepared, housing_labels)

voting_reg = VotingRegressor(
    estimators=[('ex1',ex1_reg), ('ex2',ex2_reg),('ex3',ex3_reg)])

voting_reg.fit(housing_prepared, housing_labels)



In [ ]:
from sklearn.model_selection import cross_val_score 
temp=[]
for reg in ( ex1_reg, ex2_reg,ex3_reg, voting_reg):
    reg_score=cross_val_score(voting_reg,housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
    score = np.sqrt(-reg_score)
    temp.append(score)
    display_scores(score)

 

# GradientBoost

In [15]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

reg = GradientBoostingRegressor(random_state=42,n_estimators=300)
reg.fit(housing_prepared, housing_labels)

reg_score=cross_val_score(reg,housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
score = np.sqrt(-reg_score)
display_scores(score)

Scores: [130004.59125339 139323.70853095 120131.36368748 124834.54339411
 131087.78209124 109767.76265585 114172.19319994 115930.63993785
 116088.43610888 112460.62169026]
Mean: 121380.1642549944
Standard deviation: 9100.232214799817


In [16]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

reg = GradientBoostingRegressor(random_state=42,n_estimators=500)
reg.fit(housing_prepared, housing_labels)

reg_score=cross_val_score(reg,housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
score = np.sqrt(-reg_score)
display_scores(score)

Scores: [128157.63789574 137474.62987423 116205.74095711 121612.10358556
 126779.45821955 107045.32628859 109616.4484432  113217.63470883
 114453.0638022  110892.94990436]
Mean: 118545.49936793605
Standard deviation: 9214.441997592598


In [18]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

reg = GradientBoostingRegressor(random_state=42,n_estimators=1000)
reg.fit(housing_prepared, housing_labels)

reg_score=cross_val_score(reg,housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
score = np.sqrt(-reg_score)
display_scores(score)

Scores: [126187.18505143 136031.02060184 114542.86148565 119260.42062384
 123855.80494768 104808.96927713 106909.19070706 109398.37271893
 113488.68978963 109180.46960896]
Mean: 116366.29848121472
Standard deviation: 9393.732938755158


# voting

In [17]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor

ex_reg = ExtraTreesRegressor(n_estimators=1000, random_state=42)
ex_reg.fit(housing_prepared, housing_labels)
gb_reg = GradientBoostingRegressor(random_state=42,n_estimators=1000)
gb_reg.fit(housing_prepared, housing_labels)

voting_reg = VotingRegressor(
    estimators=[('ex',ex_reg), ('gb',gb_reg)])

voting_reg.fit(housing_prepared, housing_labels)



VotingRegressor(estimators=[('ex',
                             ExtraTreesRegressor(n_estimators=1000,
                                                 random_state=42)),
                            ('gb',
                             GradientBoostingRegressor(n_estimators=1000,
                                                       random_state=42))])

In [18]:
from sklearn.model_selection import cross_val_score 
temp=[]
for reg in ( ex_reg, gb_reg, voting_reg):
    reg_score=cross_val_score(voting_reg,housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10)
    score = np.sqrt(-reg_score)
    temp.append(score)
    display_scores(score)

 

Scores: [124782.41667135 142009.63120499 115286.6029985  125707.6577528
 118636.83200339 105307.6625922  107546.69511257 105579.95473095
 110898.21326228 107106.03683915]
Mean: 116286.17031681693
Standard deviation: 11198.240288736877
Scores: [124782.41667135 142009.63120499 115286.6029985  125707.6577528
 118636.83200339 105307.6625922  107546.69511257 105579.95473095
 110898.21326228 107106.03683915]
Mean: 116286.17031681693
Standard deviation: 11198.240288736877
Scores: [124782.41667135 142009.63120499 115286.6029985  125707.6577528
 118636.83200339 105307.6625922  107546.69511257 105579.95473095
 110898.21326228 107106.03683915]
Mean: 116286.17031681693
Standard deviation: 11198.240288736877


In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor

ex_reg = ExtraTreesRegressor(n_estimators=1000, random_state=42,max_depth=30)
ex_reg.fit(housing_prepared, housing_labels)
gb_reg = GradientBoostingRegressor(random_state=42,n_estimators=2000,min_samples_split=3)
gb_reg.fit(housing_prepared, housing_labels)

voting_reg = VotingRegressor(
    estimators=[('ex',ex_reg), ('gb',gb_reg)])

voting_reg.fit(housing_prepared, housing_labels)



In [ ]:
from sklearn.model_selection import cross_val_score 
temp=[]
for reg in ( ex_reg, gb_reg, voting_reg):
    reg_score=cross_val_score(voting_reg,housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10,n_jobs=-1)
    score = np.sqrt(-reg_score)
    temp.append(score)
    display_scores(score)

 